In [ ]:
%matplotlib inline

In [ ]:
import cv2 

import sys
sys.path.append("/home/jasonraiti/Documents/GitHub/USC_REU/Project_Files/Jasons_Functions/")

from erosion_dilation_from_array import * 

from jasons_skeletonize_from_array import * 

from trim_edges import * # new_array = trim_edges(path,weight_threshold)

from erosion_dilation_from_array import *

# from generate_waypoints import *

from inverse_skeletonize_from_array import *

from overlay_images import *

from zig_zag_full_image_2 import *

from write_chinese_post_man_csv import *

from chinese_post_man import *




In [ ]:
# erosion dilation 
# WARNING : important edge case is when your skeleton has a loop in it, this may mess up all of the algorithms pretty bad 

# path to image
path = r'../MAPS/Map_originals/Ibrahim_Test/ibrahim_test_bw.png'
image = open_image(path)
option = 2 # 1 = erode first 2 = dialate first 
num_erosions = 4
num_dilations = 4

e_d_image  = erosion_dilation_from_array(image,option,num_erosions,num_dilations)

cv2.imwrite('e_d_image.png', e_d_image)



In [ ]:
# skeletonize 
med_axis , skeleton , skeleton_lee , thinned , thinned_partial = jasons_skeletonize_from_array(e_d_image)

skeletons = [med_axis , skeleton , skeleton_lee , thinned]

In [ ]:
#try and preprocess the skeletons a bit and trim edges
trimmed_skeletons = []
weight_threshold = 42
for skel in skeletons:
    
    
    # option = 2 # 1 = erode first 2 = dialate first 
    # num_erosions = 1
    # num_dilations = 1

    # skel  = erosion_dilation_from_array(skel,option,num_erosions,num_dilations)
    # show_image(skel)
    
    cv.imwrite('temp_skel_img.png', skel)

    new_array , new_image = trim_edges('temp_skel_img.png',weight_threshold)
    trimmed_skeletons.append(new_image)

In [ ]:
lotsa_inverse_skeletons = []
#inverse skeletonize trimmed graphs 
for trim_skel in trimmed_skeletons:


    med_axis , skeleton , skeleton_lee , thinned , thinned_partial = inverse_skeletonize_from_array(trim_skel)

    inverse_skeletons = [med_axis , skeleton , skeleton_lee , thinned]

    lotsa_inverse_skeletons.append(inverse_skeletons)
# for i in range(0,len(lotsa_inverse_skeletons)):
#     print("heres a skeleton")
#     show_image(trimmed_skeletons[i])

#     # for inv_skel in skels:
#     #     show_image(inv_skel)
#     show_image(lotsa_inverse_skeletons[i][0]>0) # --------------------------- Seems like medial axis is the only really good inverse skeleton




In [ ]:
#overlay boundary and one inverse skeleton image

image1 = e_d_image

image2 = (lotsa_inverse_skeletons[3][0]>0)*255
show_image(image1)

show_image(image2)

make_negative_1 = False
make_negative_2 = True

new_image = overlay_images (image1,image2,make_negative_1, make_negative_2)

show_image(new_image)





In [ ]:
#combine boundary and skeleton images to create full boundary image 

image1 = new_image

image2 = trimmed_skeletons[3]

show_image(image1)

show_image(image2)

make_negative_1 = False
make_negative_2 = True

new_image = overlay_images (image1,image2,make_negative_1, make_negative_2)

show_image(new_image)

skel_name = 'zig_zag_skeleton.png'
bound_name = 'overlay_boundary_image.png'

cv2.imwrite( skel_name, trimmed_skeletons[3] )
cv2.imwrite( bound_name, new_image*255 )


In [ ]:
#create full zig zag, this time with modified zig zag that prevents overlapping paths

path_to_skeleton = skel_name

path_to_boundary_image = bound_name

zig_zag_width = 8

image = zig_zag_full_image_2 ( path_to_skeleton, path_to_boundary_image, zig_zag_width) 

show_image(image)

zig_zag_name = 'zigzag_full.png'

cv2.imwrite(zig_zag_name, image )

In [ ]:
#create csv file from zig zag full

# option = 1 
# data = r'./write_chinese_post_man_csv_test/zigzag_full.png'
# name = 'test.csv'

option = 2
data = zig_zag_name
data = open_image(data) # for option 2 
name = 'test.csv'

path_to_csv = write_chinese_post_man_csv(data, option, name)

print(path_to_csv)

In [ ]:
#use chinese post man to calculate most efficient route covering all edges 
#HEADS UP : you will need to make a custom config file for each map you apply this to , as well as pick some arbitrary launch point (jakes landing )

# PRAISE BE TO https://github.com/brooksandrew/postman_problems#python where i got the bones for this code from


#inputs
path_to_skel = zig_zag_name
config_file  = r"/home/jasonraiti/Documents/GitHub/USC_REU/Project_Files/ibrahim_find_starting_node_config.wf" #config file 
launch_point = (34.02675, -81.2253)


circuit, graph = chinese_post_man(path_to_skel,config_file,launch_point, path_to_csv)

# print solution route
for e in circuit:
    print(e)

# print solution summary stats
for k, v in calculate_postman_solution_stats(circuit).items():
    print(k, v)


In [19]:
#parsing data from cpm 
for e in circuit:
    print(type(e))
    print(e[0])
    print(e[1])
    break

<class 'tuple'>
123
122


In [33]:
#calulate the distance in meters of your graph 

from get_graph_distance import *

waypoint1 = int(e[0])
waypoint2 = int(e[1])

graph = skeleton_to_graph(zig_zag_name)
nodes = graph.nodes()

print ( nodes[waypoint1]['o'] )
x = int(nodes[waypoint1]['o'][0])
y = int(nodes[waypoint1]['o'][1])
waypoint1 = convert_xy_to_latlong(x, y, config_file)
# waypoint1 = convert_xy_to_latlong(x, y, 'ibrahim_find_starting_node_config.wf')



print ( nodes[waypoint2]['o'] )
x = int(nodes[waypoint2]['o'][0])
y = int(nodes[waypoint2]['o'][1])
waypoint2 = convert_xy_to_latlong(x, y, config_file)
# waypoint2 = convert_xy_to_latlong(x, y, 'ibrahim_find_starting_node_config.wf')

print(waypoint1)
print(waypoint2)
path = zig_zag_name

get_graph_distance(waypoint1,waypoint2,path)


found image at :  zigzag_full.png

skeleton converted to graph

[255. 285.]
[256.25 275.25]
(34.02483136645962, -81.22806209726444)
(34.02554254658385, -81.22797598784194)

from generate_waypoints: be sure that waypoint 1 and 2 are adjacent aka have either same longitude or latitude 

found image at :  zigzag_full.png

skeleton converted to graph



total distance of graph at  zigzag_full.png  

(in meters) is 88220.37016041715


88220.37016041715

In [ ]:
# next pipeline make the thing so it is only like 10km in length 

In [ ]:
#convert coordinates to waypoints 



In [ ]:
#try and implement rural post man 